In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Get the same results each time
np.random.seed(0)

In [5]:
# Load the training data
data = pd.read_csv("/content/drive/MyDrive/UrfuMagistracy/NNEthics/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

In [6]:
comments.shape

(90902,)

### Task 1/2

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(comments)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

In [9]:
X_train.shape

(63631, 68015)

In [10]:
X_test.shape

(27271, 68015)

### Task 3

In [12]:
model = LogisticRegression(max_iter=2000)

In [13]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=2000)

In [14]:
y_pred = model.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)

In [16]:
accuracy

0.9277620915991346

### Task 4